In [1]:
import sys
import os
import datetime as dt
import math
import matplotlib as plt
import pathlib

sys.path.append(str(pathlib.Path().absolute())+"\..\..\DATA")
sys.path.append(str(pathlib.Path().absolute())+"\..\..\Crawler")
from functions import *

d:\Desktop\Github\Trading_bot\env\env01\lib\site-packages\swagger_spec_validator\validator20.py:49: SwaggerValidationWarning: Found "$ref: #/definitions/UserPreferences" with siblings that will be overwritten. See https://stackoverflow.com/a/48114924 for more information. (path #/definitions/User/properties/preferences)
  warnings.warn(
d:\Desktop\Github\Trading_bot\env\env01\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# # 做多: 當(今天短週期均線 > 今天長週期均線) 且 (昨天短週期均線 < 昨天長週期均線)
# # 做空: 當(今天短週期均線 < 今天長週期均線) 且 (昨天短週期均線 > 昨天長週期均線)
# signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
# signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())

# signal_long.astype(int).plot()
# (-signal_short.astype(int)).plot()

# # combine long and short signal
# signal = signal_long.copy()
# signal[signal_short] = -1

In [3]:
from backtesting import Backtest
from backtesting.lib import SignalStrategy
import pandas as pd

In [6]:
coin_txt = open("../../DATA/coin.txt","r",encoding = 'utf8')
coin = coin_txt.read()
coin_txt.close()

DB = DB_table(db_file_path = "../../DATA/test.db", coin_name = coin)
DB.init_db()

df = DB.get_all()
df.set_index('Timestamp', inplace=True)

# re_df = df["2021-09-13 23:00:00":]
# print(re_df)

In [7]:
class Strategy(SignalStrategy):
    def init(self):
        super().init()
        
        # Precompute the two moving averages
        close = pd.Series(self.data.Close)
        sma1 = close.rolling(20).mean()
        sma2 = close.rolling(60).mean()
        
        # Precompute signal
        signal_long = (sma1 > sma2) & (sma1.shift() < sma2.shift())
        signal_short = (sma1 < sma2) & (sma1.shift() > sma2.shift())
        
        signal = signal_long
        signal[signal_short] = -1
        
        self.set_signal(signal)
        
        
    def next(self):
        super().next()

In [8]:
bt = Backtest(df, Strategy, cash=1000000)
bt.run()
bt.plot()

Row(id='1475', ...)